In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys

Mounted at /content/gdrive/


In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/nh-daycon/'
!ls

/content/gdrive/My Drive/Colab Notebooks/nh-daycon
nh.ipynb  open


In [46]:
%matplotlib inline

width = 6
height = 3
import matplotlib
matplotlib.rcParams['figure.figsize'] = [width, height]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
import pdb

import tensorflow as tf

import sklearn
import sklearn.preprocessing

import numpy as np
import pandas as pd

from scipy.stats import pearsonr, spearmanr

import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [47]:
import pandas as pd


In [71]:
cus = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/nh-daycon/open/cus_info.csv")
train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/nh-daycon/open/stk_hld_train.csv")
iem = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/nh-daycon/open/iem_info_20210902.csv")
submission  = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/nh-daycon/open/sample_submission.csv")
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/nh-daycon/open/stk_hld_test.csv")
hist = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/nh-daycon/open/stk_bnc_hist.csv")

In [72]:
train["hist_d"] = train["hold_d"]*0.6
train.hist_d = np.trunc(train["hist_d"])

In [73]:
#left조인 왼쪽 테이블의 한개의 레코드에 여러개의 오른쪽 테이블 레코드가 일치할 경우, 해당 왼쪽 레코드를 여러번 표시하게 된다.


train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

In [74]:
train_data.head()
#byn_dt : 매수일자
#hold_d : 보유기간
#stk_dit_cd : 시장구분

,act_id,iem_cd,byn_dt,hold_d,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99


In [75]:
train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)
#labe이 될 Y를 때어낸다.
train_data.head()

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99


In [76]:
train_label.head()

0     11
1     80
2      5
3     22
4    324
Name: hold_d, dtype: int64

In [77]:
hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
# 주식 현재가 = 평가금액 / 잔고수량
# 현재가 X 보유 주식수 = 평가금액
hist = hist.fillna(0)
hist.head()
#hist : 일별 종목 잔고수량 및 금액, 액면가 정보
#stk_par_pr = 액면가
#stk_p : 기준일자(bse_dt) 에서의 종목 현재가

,act_id,bse_dt,iem_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200820,A008770,40.0,2828000.0,5000.0,70700.0
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A008770,20.0,1390000.0,5000.0,69500.0
2,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20160104,A005940,311.0,2982490.0,5000.0,9590.0
3,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200814,A005930,40.0,2320000.0,100.0,58000.0
4,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,20200623,A005930,20.0,1028000.0,100.0,51400.0


In [78]:
train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data.head(10)
#첫번째사람 기준 일자 : 20180726에 잔고수량274 ->  20180810에 잔고수량0.0 = 보유기간 11일

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180807,174.0,7708200.0,5000.0,44300.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180809,180.0,7866000.0,5000.0,43700.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180810,0.0,0.0,5000.0,0.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20200116,208.0,6229600.0,5000.0,29950.0
5,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20200120,0.0,0.0,5000.0,0.0
6,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
7,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180314,5.0,12940000.0,5000.0,2588000.0
8,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180419,11.0,29029000.0,5000.0,2639000.0
9,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180420,10.0,25810000.0,5000.0,2581000.0


In [79]:
#byn_dt : 매수일자
#bse_dt : 국내주식 잔고 이력에서의 기준일자
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
#매수일자와 기준일자(잔고수량이 변경될때마다 기록된 값)가 같은것만 남겨둔다.
train_data.head()


,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0
6,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
15,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0
18,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1,20201112,60.0,462000.0,500.0,7700.0
20,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0


In [80]:
train_label.head()
#초기에  

0     11
1     80
2      5
3     22
4    324
Name: hold_d, dtype: int64

In [81]:
train_data.reset_index(drop = True, inplace = True)
train_data.head()

,act_id,iem_cd,byn_dt,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1,20201112,60.0,462000.0,500.0,7700.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0


In [82]:
#test_data에도 같이 실행시킨다 
test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

In [83]:
test_data.head()

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1,20200522,300.0,3945000.0,5000.0,13150.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99,20190823,198.0,2524500.0,500.0,12750.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99,20200611,138.0,4291800.0,500.0,31100.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,파인텍,13,3,99,20200120,2972.0,6716720.0,500.0,2260.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2,20201217,10.0,464500.0,100.0,46450.0


In [84]:
train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
#이렇게 함으로서 훈련 데이터에서 학습해야할 특성들을 명세화한다.
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)
train_data.head()

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,274.0,11782000.0,5000.0,43000.0
1,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,2.0,4990000.0,5000.0,2495000.0
2,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99,786.0,14619600.0,1000.0,18600.0
3,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1,60.0,462000.0,500.0,7700.0
4,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99,183.0,8125200.0,5000.0,44400.0


In [85]:
train_data.shape


(681472, 18)

In [86]:
test_data.head()
#여기서에 hist_d는 과거 보유일

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,153,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1,300.0,3945000.0,5000.0,13150.0
1,335,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99,198.0,2524500.0,500.0,12750.0
2,139,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99,138.0,4291800.0,500.0,31100.0
3,236,1,9,3,2,2,9,5,5,8,파인텍,13,3,99,2972.0,6716720.0,500.0,2260.0
4,9,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2,10.0,464500.0,100.0,46450.0


In [87]:
test_data.head()

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,153,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1,300.0,3945000.0,5000.0,13150.0
1,335,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99,198.0,2524500.0,500.0,12750.0
2,139,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99,138.0,4291800.0,500.0,31100.0
3,236,1,9,3,2,2,9,5,5,8,파인텍,13,3,99,2972.0,6716720.0,500.0,2260.0
4,9,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2,10.0,464500.0,100.0,46450.0


In [88]:
test_data.shape

(70596, 18)

In [89]:






L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

In [90]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [91]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds.
[500]	valid_0's rmse: 4.22398	valid_0's l2: 17.842
[1000]	valid_0's rmse: 4.07726	valid_0's l2: 16.624
[1500]	valid_0's rmse: 4.04283	valid_0's l2: 16.3445
[2000]	valid_0's rmse: 4.01963	valid_0's l2: 16.1574
Did not meet early stopping. Best iteration is:
[1998]	valid_0's rmse: 4.01939	valid_0's l2: 16.1555
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's rmse: 1.29387	valid_0's l2: 1.67409
[1000]	valid_0's rmse: 1.29113	valid_0's l2: 1.66703
Early stopping, best iteration is:
[733]	valid_0's rmse: 1.18197	valid_0's l2: 1.39706
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's rmse: 0.682073	valid_0's l2: 0.465223
[1000]	valid_0's rmse: 0.480327	valid_0's l2: 0.230714
Early stopping, best iteration is:
[764]	valid_0's rmse: 0.478875	valid_0's l2: 0.229322
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's rmse: 0.574113	valid_0's l2: 0.3

In [92]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [94]:
print(predict)

[254.86217177 557.6633586  232.32522676 ... 876.32289663  18.99816083
   7.46477122]
